In [1]:
import numpy as np
import pandas as pd
import math
from scipy.special import factorial
import random
import warnings
warnings.filterwarnings("ignore")
random.seed(42)

dataframes_list = []
df = pd.read_excel('Project_Customer_Demand.xlsx')
df1 = pd.read_excel('Project_Customer_Demand.xlsx','Sample')
for j in range(1,3,1):
    df_newsample = df.copy(deep=True)
    for i in range(2,26,1):
        df_newsample['Unnamed: 1'].iloc[i] = random.choice(np.random.poisson(df['Unnamed: 1'].iloc[i],10))
        df_newsample['Unnamed: 2'].iloc[i] = random.choice(np.random.poisson(df['Unnamed: 2'].iloc[i],10))
    dataframes_list.append(df_newsample)

dataframes_list.insert(0,df)
dataframes_list.insert(1,df1)
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df.drop(index=0,inplace=True)
    temp_df.columns = temp_df.iloc[0]
    temp_df.drop(index=1,inplace=True)
    temp_df.reset_index(drop=True)
    temp_df.loc[-1] = [0,0,0]
    temp_df = temp_df.sort_index().reset_index(drop=True)
    temp_df.loc[-1] = [0,0,0]
    temp_df = temp_df.sort_index().reset_index(drop=True)
    temp_df.loc[-1] = [0,0,0]
    temp_df = temp_df.sort_index().reset_index(drop=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30 #Intial inventory at A
    temp_df['Avail B ending'][0:3] = 30 #Intial inventory at B
    temp_df['Mov a2b'] = 0 #Inventory moved from A to B
    temp_df['Mov b2a'] = 0 #Inventory moved from A to B
    temp_df['cust a'] = 0 #Customers satisfied at Station A
    temp_df['cust b'] = 0 #Customers satisfied at Station B
    temp_df['Transit A'] = 0 #Vehicles moving from A to B
    temp_df['Transit B'] = 0 #Vehicles moving from B to A
    temp_df['veh in sys'] = 0 #Total vehicles in the system
    temp_df = temp_df.rename(columns={np.nan:'Hour'})

    dataframes_list[i] = temp_df.copy(deep=True)
    
Ultab = 3 #(T+Uab) #Transit time from A to B
Ultba = 4 #(T+Uba) #Transit time from B to A
sigABltr = 1.732 #sqrt(Uab+t)
sigBAltr = 2 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.96
nbar_s = 1.969
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

Profit = 0

In [2]:
dataframes_list[1].iloc[3:,:]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
3,00:00:00,6,3,0,0,0,0,0,0,0,0,0,0,0
4,01:00:00,2,4,0,0,0,0,0,0,0,0,0,0,0
5,02:00:00,5,3,0,0,0,0,0,0,0,0,0,0,0
6,03:00:00,4,0,0,0,0,0,0,0,0,0,0,0,0
7,04:00:00,0,2,0,0,0,0,0,0,0,0,0,0,0
8,05:00:00,1,3,0,0,0,0,0,0,0,0,0,0,0
9,06:00:00,19,9,0,0,0,0,0,0,0,0,0,0,0
10,07:00:00,14,11,0,0,0,0,0,0,0,0,0,0,0
11,08:00:00,19,6,0,0,0,0,0,0,0,0,0,0,0
12,09:00:00,21,10,0,0,0,0,0,0,0,0,0,0,0


# No policy

In [4]:
for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3])
        temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2])
        temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3])  
        temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2])  
        temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
        temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
        temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
        temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
MovB = [] 
cus_sata = []
cus_satb = []
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    da = 0
    db = 0
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da = dataframes_list[i]['Station A to B'].sum()
    db = dataframes_list[i]['Station B to A'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_nopol = 0
Profit_nopol = sum(Profit)/len(Profit)
Pmean_nopol = 0
Pmean_nopol = Profit[0]
cusa_nopol = sum(cus_sata)/len(cus_sata)
cusb_nopol = sum(cus_satb)/len(cus_satb)
sample_nopol =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [5]:
temp = dataframes_list[1].reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending','cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'])
temp.iloc[3:,:]

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,cust a,cust b,Transit A,Transit B,veh in sys
3,00:00:00,6,3,30,24,30,27,6,3,6,3,60
4,01:00:00,2,4,24,22,27,23,2,4,8,7,60
5,02:00:00,5,3,22,17,29,26,5,3,7,10,60
6,03:00:00,4,0,20,16,28,28,4,0,9,7,60
7,04:00:00,0,2,20,20,33,31,0,2,4,5,60
8,05:00:00,1,3,23,22,35,32,1,3,1,5,60
9,06:00:00,19,9,22,3,32,23,19,9,20,14,60
10,07:00:00,14,11,5,0,24,13,5,11,24,23,60
11,08:00:00,19,6,3,0,32,26,3,6,8,26,60
12,09:00:00,21,10,9,0,31,21,9,10,12,27,60


In [6]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)


In [7]:
dataframes_list[2]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,2,6,0,0,0,0,0,0,0,0,0,0,0
4,01:00:00,0,4,0,0,0,0,0,0,0,0,0,0,0
5,02:00:00,6,4,0,0,0,0,0,0,0,0,0,0,0
6,03:00:00,5,0,0,0,0,0,0,0,0,0,0,0,0
7,04:00:00,2,0,0,0,0,0,0,0,0,0,0,0,0
8,05:00:00,1,2,0,0,0,0,0,0,0,0,0,0,0
9,06:00:00,9,6,0,0,0,0,0,0,0,0,0,0,0


In [8]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Confidence interval - 95% and review period - 1 hour(S,T)

In [9]:

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
                                                                                                                            
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
        
    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_95_hr1 = 0
Profit_95_hr1 = sum(Profit)/len(Profit)
Pmean_95_hr1 = 0
Pmean_95_hr1 = Profit[0]
cusa_95_hr1 = sum(cus_sata)/len(cus_sata)
cusb_95_hr1 = sum(cus_satb)/len(cus_satb)
sample_95_hr1 =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [10]:
sample_95_hr1

17260

In [11]:
dataframes_list[0]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,...,Transit B,veh in sys,A2B U_DLTR,A2B Sig_DLTR,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba
0,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
3,00:00:00,5,4,25,30,30,26,0,0,5,...,4,60,15,3.87287,16,4.0,23,24,11,10
4,01:00:00,2,3,23,25,26,23,0,0,2,...,7,60,6,2.449418,12,3.464102,11,19,6,9
5,02:00:00,4,2,19,23,28,26,0,0,4,...,9,60,12,3.464,8,2.828427,19,14,9,7
6,03:00:00,3,1,20,23,28,27,0,0,3,...,6,60,9,2.999912,4,2.0,15,8,8,5
7,04:00:00,1,2,22,23,31,29,0,0,1,...,5,60,3,1.732,8,2.828427,7,14,4,7
8,05:00:00,1,5,23,24,32,27,0,0,1,...,8,60,3,1.732,20,4.472136,7,29,4,12
9,06:00:00,17,11,7,24,28,17,0,0,17,...,18,60,51,7.141219,44,6.63325,65,58,23,19


In [12]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,24,30,30,27,0,0,6,3,6,3,60
4,01:00:00,2,4,22,24,27,23,0,0,2,4,8,7,60
5,02:00:00,5,3,17,22,29,26,0,0,5,3,7,10,60
6,03:00:00,4,0,16,20,28,28,0,0,4,0,9,7,60
7,04:00:00,0,2,20,20,33,31,0,0,0,2,4,5,60
8,05:00:00,1,3,22,23,35,32,0,0,1,3,1,5,60
9,06:00:00,19,9,3,22,32,23,0,0,19,9,20,14,60


In [13]:
merge = pd.merge(dataframes_list[1],df_mean[['Hour','Sab','Sba']])
merge = merge.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Sab', 'Sba', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'
                                                      ])                                                 
merge.iloc[9:,:]   

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Sab,Sba,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
9,00:00:00,6,3,30,24,30,27,23,24,0,0,6,3,6,3,60
10,01:00:00,2,4,24,22,27,23,11,19,0,0,2,4,8,7,60
11,02:00:00,5,3,22,17,29,26,19,14,0,0,5,3,7,10,60
12,03:00:00,4,0,20,16,28,28,15,8,0,0,4,0,9,7,60
13,04:00:00,0,2,20,20,33,31,7,14,0,0,0,2,4,5,60
14,05:00:00,1,3,23,22,35,32,7,29,0,0,1,3,1,5,60
15,06:00:00,19,9,22,3,32,23,65,58,0,0,19,9,20,14,60
16,07:00:00,14,11,5,0,24,13,62,48,0,0,5,11,24,23,60
17,08:00:00,19,6,3,0,32,26,76,39,0,0,3,6,8,26,60
18,09:00:00,21,10,9,0,31,21,69,53,0,0,9,10,12,27,60


In [14]:
merge = pd.merge(dataframes_list[2],df_mean[['Hour','Sab','Sba']])
merge = merge.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Sab', 'Sba', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'
                                                      ])                                                 
merge.iloc[9:,:]   

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Sab,Sba,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
9,00:00:00,2,6,30,28,30,24,23,24,0,0,2,6,2,6,60
10,01:00:00,0,4,28,28,24,20,11,19,0,0,0,4,2,10,60
11,02:00:00,6,4,28,22,22,18,19,14,0,0,6,4,6,14,60
12,03:00:00,5,0,28,23,18,18,15,8,0,0,5,0,11,8,60
13,04:00:00,2,0,27,25,24,24,7,14,0,0,2,0,7,4,60
14,05:00:00,1,2,29,28,29,27,7,29,0,0,1,2,3,2,60
15,06:00:00,9,6,28,19,29,23,65,58,0,0,9,6,10,8,60
16,07:00:00,19,7,19,0,24,17,62,48,0,0,19,7,28,15,60
17,08:00:00,21,4,2,0,26,22,76,39,0,0,2,4,21,17,60
18,09:00:00,15,11,6,0,41,30,69,53,0,0,6,11,8,22,60


In [15]:
sample_95_hr1

17260

In [17]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

# Average policy 95%confidence and review period - 1 hour

In [18]:

Ultab = 3 #(T+Uab)
Ultba = 4 #(T+Uba)
sigABltr = 1.732 #sqrt(Uab+t)
sigBAltr = 2 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.96
nbar_s = 1.969
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

Profit = 0

#%%time

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
                                                                                                                            
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)
    
    
print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg95_hr1 = 0
Profit_avg95_hr1 = sum(Profit)/len(Profit)
Pmean_avg95_hr1 = 0
Pmean_avg95_hr1 = Profit[0]
cusa_avg95_hr1 = sum(cus_sata)/len(cus_sata)
cusb_avg95_hr1 = sum(cus_satb)/len(cus_satb)
sample_avg95_hr1 =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [19]:
dataframes_list[0].columns

Index(['Hour', 'Station A to B', 'Station B to A', 'Avail A ending',
       'Avail A starting', 'Avail B starting', 'Avail B ending', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys',
       'A2B U_DLTR', 'A2B Sig_DLTR', 'B2A U_DLTR', 'B2A Sig_DLTR', 'Sab',
       'Sba', 'Avg inv ab', 'Avg inv ba'],
      dtype='object', name=1)

In [20]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,24,30,30,27,0,0,6,3,6,3,60
4,01:00:00,2,4,22,24,27,23,0,0,2,4,8,7,60
5,02:00:00,5,3,17,22,29,26,0,0,5,3,7,10,60
6,03:00:00,4,0,16,20,28,28,0,0,4,0,9,7,60
7,04:00:00,0,2,20,20,33,31,0,0,0,2,4,5,60
8,05:00:00,1,3,22,23,35,32,0,0,1,3,1,5,60
9,06:00:00,19,9,3,22,32,23,0,0,19,9,20,14,60


In [21]:
dataframes_list[0]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,...,Transit B,veh in sys,A2B U_DLTR,A2B Sig_DLTR,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba
0,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
3,00:00:00,5,4,25,30,30,26,0,0,5,...,4,60,15,3.87287,16,4.0,23,24,11,10
4,01:00:00,2,3,23,25,26,23,0,0,2,...,7,60,6,2.449418,12,3.464102,11,19,6,9
5,02:00:00,4,2,19,23,28,26,0,0,4,...,9,60,12,3.464,8,2.828427,19,14,9,7
6,03:00:00,3,1,20,23,28,27,0,0,3,...,6,60,9,2.999912,4,2.0,15,8,8,5
7,04:00:00,1,2,22,23,31,29,0,0,1,...,5,60,3,1.732,8,2.828427,7,14,4,7
8,05:00:00,1,5,23,24,32,27,0,0,1,...,8,60,3,1.732,20,4.472136,7,29,4,12
9,06:00:00,17,11,7,24,28,17,0,0,17,...,18,60,51,7.141219,44,6.63325,65,58,23,19


In [22]:
merge2 = pd.merge(dataframes_list[1],df_mean[['Hour','Avg inv ab','Avg inv ba']])
merge2 = merge2.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Avg inv ab', 'Avg inv ba', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'
                                                      ])                                                 
merge2.iloc[9:,:]   

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Avg inv ab,Avg inv ba,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
9,00:00:00,6,3,30,24,30,27,11,10,0,0,6,3,6,3,60
10,01:00:00,2,4,24,22,27,23,6,9,0,0,2,4,8,7,60
11,02:00:00,5,3,22,17,29,26,9,7,0,0,5,3,7,10,60
12,03:00:00,4,0,20,16,28,28,8,5,0,0,4,0,9,7,60
13,04:00:00,0,2,20,20,33,31,4,7,0,0,0,2,4,5,60
14,05:00:00,1,3,23,22,35,32,4,12,0,0,1,3,1,5,60
15,06:00:00,19,9,22,3,32,23,23,19,0,0,19,9,20,14,60
16,07:00:00,14,11,5,0,24,13,22,17,0,0,5,11,24,23,60
17,08:00:00,19,6,3,0,32,26,26,14,0,0,3,6,8,26,60
18,09:00:00,21,10,9,0,31,21,24,18,0,0,9,10,12,27,60


In [23]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)


# Confidence interval - 90% and review period - 2 hours (S,T) policy# 

In [24]:

    
Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.64
nbar_s = 1.661
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

#%%time

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            temp_df['Mov b2a'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            temp_df['Mov a2b'].iloc[i]  = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_90_hr2 = 0
Profit_90_hr2 = sum(Profit)/len(Profit)
Pmean_90_hr2 = 0
Pmean_90_hr2 = Profit[0]
cusa_90_hr2 = sum(cus_sata)/len(cus_sata)
cusb_90_hr2 = sum(cus_satb)/len(cus_satb)
sample_90_hr2 =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [25]:
dataframes_list[0]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,...,Transit B,veh in sys,A2B U_DLTR,A2B Sig_DLTR,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba
0,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
3,00:00:00,5,4,25,30,30,26,0,0,5,...,4,60,20,4.472136,20,4.472,28,28,10,10
4,01:00:00,2,3,23,25,26,23,0,0,2,...,7,60,8,2.828427,15,3.872866,13,22,6,8
5,02:00:00,4,2,19,23,28,26,0,0,4,...,9,60,16,4.0,10,3.162182,23,16,9,7
6,03:00:00,3,1,20,23,28,27,0,0,3,...,6,60,12,3.464102,5,2.236,18,9,8,5
7,04:00:00,1,2,22,23,31,29,0,0,1,...,5,60,4,2.0,10,3.162182,8,16,4,7
8,05:00:00,1,5,23,24,32,27,0,0,1,...,8,60,4,2.0,25,4.999848,8,34,4,11
9,06:00:00,17,11,7,24,28,17,0,0,17,...,18,60,68,8.246211,55,7.415973,82,68,23,18


In [26]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,24,30,30,27,0,0,6,3,6,3,60
4,01:00:00,2,4,22,24,27,23,0,0,2,4,8,7,60
5,02:00:00,5,3,17,22,29,26,0,0,5,3,7,10,60
6,03:00:00,4,0,16,20,28,28,0,0,4,0,9,7,60
7,04:00:00,0,2,20,20,33,31,0,0,0,2,4,5,60
8,05:00:00,1,3,22,23,35,32,0,0,1,3,1,5,60
9,06:00:00,19,9,3,22,32,23,0,0,19,9,20,14,60


In [27]:
merge = pd.merge(dataframes_list[2],df_mean[['Hour','Sab','Sba']])
merge = merge.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Sab', 'Sba', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'
                                                      ])                                                 
merge.iloc[9:,:]   

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Sab,Sba,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
9,00:00:00,2,6,30,28,30,24,28,28,0,0,2,6,2,6,60
10,01:00:00,0,4,28,28,24,20,13,22,0,0,0,4,2,10,60
11,02:00:00,6,4,28,22,22,18,23,16,0,0,6,4,6,14,60
12,03:00:00,5,0,28,23,18,18,18,9,0,0,5,0,11,8,60
13,04:00:00,2,0,27,25,24,24,8,16,0,0,2,0,7,4,60
14,05:00:00,1,2,29,28,29,27,8,34,0,0,1,2,3,2,60
15,06:00:00,9,6,28,19,29,23,82,68,0,0,9,6,10,8,60
16,07:00:00,19,7,19,0,24,17,78,57,0,0,19,7,28,15,60
17,08:00:00,21,4,2,0,26,22,95,45,0,0,2,4,21,17,60
18,09:00:00,15,11,6,0,41,30,86,62,0,0,6,11,8,22,60


In [28]:
dataframes_list[2]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,2,6,28,30,30,24,0,0,2,6,2,6,60
4,01:00:00,0,4,28,28,24,20,0,0,0,4,2,10,60
5,02:00:00,6,4,22,28,22,18,0,0,6,4,6,14,60
6,03:00:00,5,0,23,28,18,18,0,0,5,0,11,8,60
7,04:00:00,2,0,25,27,24,24,0,0,2,0,7,4,60
8,05:00:00,1,2,28,29,29,27,0,0,1,2,3,2,60
9,06:00:00,9,6,19,28,29,23,0,0,9,6,10,8,60


In [29]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)
    

# Average policy 90%confidence and review period - 2 hour

In [30]:

Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.64
nbar_s = 1.661
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if  (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                                 
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg90_hr2 = 0
Profit_avg90_hr2 = sum(Profit)/len(Profit)
Pmean_avg90_hr2 = 0
Pmean_avg90_hr2 = Profit[0]
cusa_avg90_hr2 = sum(cus_sata)/len(cus_sata)
cusb_avg90_hr2 = sum(cus_satb)/len(cus_satb)
sample_avg90_hr2 =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

# Confidence interval - 90% and review period - 1 hour (S,T) policy

In [31]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

In [32]:
dataframes_list[0]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,...,Transit B,veh in sys,A2B U_DLTR,A2B Sig_DLTR,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba
0,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
3,00:00:00,5,4,0,0,0,0,0,0,0,...,0,0,20,4.472136,20,4.472,28,28,10,10
4,01:00:00,2,3,0,0,0,0,0,0,0,...,0,0,8,2.828427,15,3.872866,13,22,6,8
5,02:00:00,4,2,0,0,0,0,0,0,0,...,0,0,16,4.0,10,3.162182,23,16,9,7
6,03:00:00,3,1,0,0,0,0,0,0,0,...,0,0,12,3.464102,5,2.236,18,9,8,5
7,04:00:00,1,2,0,0,0,0,0,0,0,...,0,0,4,2.0,10,3.162182,8,16,4,7
8,05:00:00,1,5,0,0,0,0,0,0,0,...,0,0,4,2.0,25,4.999848,8,34,4,11
9,06:00:00,17,11,0,0,0,0,0,0,0,...,0,0,68,8.246211,55,7.415973,82,68,23,18


In [33]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,0,0,0,0,0,0,0,0,0,0,0
4,01:00:00,2,4,0,0,0,0,0,0,0,0,0,0,0
5,02:00:00,5,3,0,0,0,0,0,0,0,0,0,0,0
6,03:00:00,4,0,0,0,0,0,0,0,0,0,0,0,0
7,04:00:00,0,2,0,0,0,0,0,0,0,0,0,0,0
8,05:00:00,1,3,0,0,0,0,0,0,0,0,0,0,0
9,06:00:00,19,9,0,0,0,0,0,0,0,0,0,0,0


In [34]:
Ultab = 3 #(T+Uab)
Ultba = 4 #(T+Uba)
sigABltr = 1.732 #sqrt(Uab+t)
sigBAltr = 2 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.64
nbar_s = 1.661
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            temp_df['Mov b2a'].iloc[i]  = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])

            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            temp_df['Mov a2b'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
             
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            temp_df['Mov a2b'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            temp_df['Mov b2a'].iloc[i]  = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_90_hr1 = 0
Profit_90_hr1 = sum(Profit)/len(Profit)
Pmean_90_hr1 = 0
Pmean_90_hr1 = Profit[0]
cusa_90_hr1 = sum(cus_sata)/len(cus_sata)
cusb_90_hr1 = sum(cus_satb)/len(cus_satb)
sample_90_hr1 =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [35]:
df_mean

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,...,Transit B,veh in sys,A2B U_DLTR,A2B Sig_DLTR,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba
0,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
3,00:00:00,5,4,0,0,0,0,0,0,0,...,0,0,15,3.87287,16,4.0,22,23,9,9
4,01:00:00,2,3,0,0,0,0,0,0,0,...,0,0,6,2.449418,12,3.464102,11,18,6,8
5,02:00:00,4,2,0,0,0,0,0,0,0,...,0,0,12,3.464,8,2.828427,18,13,8,6
6,03:00:00,3,1,0,0,0,0,0,0,0,...,0,0,9,2.999912,4,2.0,14,8,7,4
7,04:00:00,1,2,0,0,0,0,0,0,0,...,0,0,3,1.732,8,2.828427,6,13,4,6
8,05:00:00,1,5,0,0,0,0,0,0,0,...,0,0,3,1.732,20,4.472136,6,28,4,10
9,06:00:00,17,11,0,0,0,0,0,0,0,...,0,0,51,7.141219,44,6.63325,63,55,21,17


In [36]:
dataframes_list[0]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,...,Transit B,veh in sys,A2B U_DLTR,A2B Sig_DLTR,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba
0,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
3,00:00:00,5,4,25,30,30,26,0,0,5,...,4,60,15,3.87287,16,4.0,22,23,9,9
4,01:00:00,2,3,23,25,26,23,0,0,2,...,7,60,6,2.449418,12,3.464102,11,18,6,8
5,02:00:00,4,2,19,23,28,26,0,0,4,...,9,60,12,3.464,8,2.828427,18,13,8,6
6,03:00:00,3,1,20,23,28,27,0,0,3,...,6,60,9,2.999912,4,2.0,14,8,7,4
7,04:00:00,1,2,22,23,31,29,0,0,1,...,5,60,3,1.732,8,2.828427,6,13,4,6
8,05:00:00,1,5,23,24,32,27,0,0,1,...,8,60,3,1.732,20,4.472136,6,28,4,10
9,06:00:00,17,11,7,24,28,17,0,0,17,...,18,60,51,7.141219,44,6.63325,63,55,21,17


In [37]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,24,30,30,27,0,0,6,3,6,3,60
4,01:00:00,2,4,22,24,27,23,0,0,2,4,8,7,60
5,02:00:00,5,3,17,22,29,26,0,0,5,3,7,10,60
6,03:00:00,4,0,16,20,28,28,0,0,4,0,9,7,60
7,04:00:00,0,2,20,20,33,31,0,0,0,2,4,5,60
8,05:00:00,1,3,22,23,35,32,0,0,1,3,1,5,60
9,06:00:00,19,9,3,22,32,23,0,0,19,9,20,14,60


In [38]:
merge = pd.merge(dataframes_list[1],df_mean[['Hour','Sab','Sba']])
merge = merge.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Sab', 'Sba', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'])                                                 
merge.iloc[9:,:]   

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Sab,Sba,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
9,00:00:00,6,3,30,24,30,27,22,23,0,0,6,3,6,3,60
10,01:00:00,2,4,24,22,27,23,11,18,0,0,2,4,8,7,60
11,02:00:00,5,3,22,17,29,26,18,13,0,0,5,3,7,10,60
12,03:00:00,4,0,20,16,28,28,14,8,0,0,4,0,9,7,60
13,04:00:00,0,2,20,20,33,31,6,13,0,0,0,2,4,5,60
14,05:00:00,1,3,23,22,35,32,6,28,0,0,1,3,1,5,60
15,06:00:00,19,9,22,3,32,23,63,55,0,0,19,9,20,14,60
16,07:00:00,14,11,5,0,24,13,60,46,0,0,5,11,24,23,60
17,08:00:00,19,6,3,0,32,26,73,37,0,0,3,6,8,26,60
18,09:00:00,21,10,9,0,31,21,67,51,0,0,9,10,12,27,60


# Average policy 90%confidence and review period - 1 hour

In [39]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)
    
    
Ultab = 3 #(T+Uab)
Ultba = 4 #(T+Uba)
sigABltr = 1.732 #sqrt(Uab+t)
sigBAltr = 2 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.64
nbar_s = 1.661
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if  (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] =  var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                          
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg90_hr1 = 0
Profit_avg90_hr1 = sum(Profit)/len(Profit)
Pmean_avg90_hr1 = 0
Pmean_avg90_hr1 = Profit[0]
cusa_avg90_hr1 = sum(cus_sata)/len(cus_sata)
cusb_avg90_hr1 = sum(cus_satb)/len(cus_satb)
sample_avg90_hr1 =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [40]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)
    

# Confidence interval - 95% and review period - 2 hour (S,T) policy

In [41]:

    
Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.96
nbar_s = 1.969
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

#%%time

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    ma = dataframes_list[i]['Mov a2b'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_95_hr2 = 0
Profit_95_hr2 = sum(Profit)/len(Profit)
Pmean_95_hr2 = 0
Pmean_95_hr2 = Profit[0]
cusa_95_hr2 = sum(cus_sata)/len(cus_sata)
cusb_95_hr2 = sum(cus_satb)/len(cus_satb)
sample_95_hr2  =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

# Average policy 95%confidence and review period - 2 hour

In [42]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)
    
Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.96
nbar_s = 1.969
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if  (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
cus_sata = []
cus_satb = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    ma = dataframes_list[i]['Mov a2b'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(dataframes_list[1])
print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg95_hr2 = 0
Profit_avg95_hr2 = sum(Profit)/len(Profit)
Pmean_avg95_hr2 = 0
Pmeanavg95_hr2 = Profit[0]
cusa_avg95_hr2 = sum(cus_sata)/len(cus_sata)
cusb_avg95_hr2 = sum(cus_satb)/len(cus_satb)
sample_avg95_hr2  =  Profit[1]

1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
0          0              0              0              30                 0   
1          0              0              0              30                 0   
2          0              0              0              30                 0   
3   00:00:00              6              3              24                30   
4   01:00:00              2              4              22                24   
5   02:00:00              5              3              17                22   
6   03:00:00              4              0              16                20   
7   04:00:00              0              2              20                20   
8   05:00:00              1              3              22                23   
9   06:00:00             19              9               3                22   
10  07:00:00             14             11               0                 5   
11  08:00:00             19             

In [43]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

# Confidence interval - 80% and review period - 1 hour (S,T) policy

In [44]:

    
    
Ultab = 3 #(T+Uab)
Ultba = 4 #(T+Uba)
sigABltr = 1.732 #sqrt(Uab+t)
sigBAltr = 2 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.28
nbar_s = 1.327
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

#%%time

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
                                                                                                                         
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
cus_sata = []
cus_satb = []
custB = [] 
Profit = []
MovA = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(sum(Profit)/len(Profit))
print(Profit)
Profit_80_hr1 = 0
Profit_80_hr1 = sum(Profit)/len(Profit)
Pmean_80_hr1 = 0
Pmean_80_hr1 = Profit[0]
cusa_80_hr1 = sum(cus_sata)/len(cus_sata)
cusb_80_hr1 = sum(cus_satb)/len(cus_satb)
sample_80_hr1  =  Profit[1]

16775.0
[17020, 17260, 15390, 17430]


In [45]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)
    

# Average policy 80%confidence and review period - 1 hour

In [46]:

    
Ultab = 3 #(T+Uab)
Ultba = 4 #(T+Uba)
sigABltr = 1.732 #sqrt(Uab+t)
sigBAltr = 2 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.28
nbar_s = 1.327
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if  (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
cus_sata = []
cus_satb = []
custB = [] 
Profit = []
MovA = []
MovB = [] 
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg80_hr1 = 0
Profit_avg80_hr1 = sum(Profit)/len(Profit)
Pmean_avg80_hr1 = 0
Pmean_avg80_hr1 = Profit[0]
cusa_avg80_hr1 = sum(cus_sata)/len(cus_sata)
cusb_avg80_hr1 = sum(cus_satb)/len(cus_satb)
sample_avg80_hr1  =  Profit[1]

16775.0
[17020, 17260, 15390, 17430]


In [47]:
merge = pd.merge(dataframes_list[1],df_mean[['Hour','Avg inv ab','Avg inv ba']])
merge = merge.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Avg inv ab', 'Avg inv ba', 'Mov a2b',
       'Mov b2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'])                                                 
merge.iloc[9:,:]   

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Avg inv ab,Avg inv ba,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
9,00:00:00,6,3,30,24,30,27,8,8,0,0,6,3,6,3,60
10,01:00:00,2,4,24,22,27,23,5,7,0,0,2,4,8,7,60
11,02:00:00,5,3,22,17,29,26,7,5,0,0,5,3,7,10,60
12,03:00:00,4,0,20,16,28,28,6,4,0,0,4,0,9,7,60
13,04:00:00,0,2,20,20,33,31,3,5,0,0,0,2,4,5,60
14,05:00:00,1,3,23,22,35,32,3,9,0,0,1,3,1,5,60
15,06:00:00,19,9,22,3,32,23,18,15,0,0,19,9,20,14,60
16,07:00:00,14,11,5,0,24,13,18,13,0,0,5,11,24,23,60
17,08:00:00,19,6,3,0,32,26,21,11,0,0,3,6,8,26,60
18,09:00:00,21,10,9,0,31,21,19,14,0,0,9,10,12,27,60


In [48]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,24,30,30,27,0,0,6,3,6,3,60
4,01:00:00,2,4,22,24,27,23,0,0,2,4,8,7,60
5,02:00:00,5,3,17,22,29,26,0,0,5,3,7,10,60
6,03:00:00,4,0,16,20,28,28,0,0,4,0,9,7,60
7,04:00:00,0,2,20,20,33,31,0,0,0,2,4,5,60
8,05:00:00,1,3,22,23,35,32,0,0,1,3,1,5,60
9,06:00:00,19,9,3,22,32,23,0,0,19,9,20,14,60


In [49]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

# Confidence interval - 80% and review period - 2 hour (S,T) policy

In [50]:
Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.28
nbar_s = 1.327
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

#%%time

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
                                                                                                                           
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
MovB = []
cus_sata = []
cus_satb = []
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(sum(Profit)/len(Profit))
print(Profit)
Profit_80_hr2 = 0
Profit_80_hr2 = sum(Profit)/len(Profit)
Pmean_80_hr2 = 0
Pmean_80_hr2 = Profit[0]
cusa_80_hr2 = sum(cus_sata)/len(cus_sata)
cusb_80_hr2 = sum(cus_satb)/len(cus_satb)
sample_80_hr2 =  Profit[1]

16775.0
[17020, 17260, 15390, 17430]


In [51]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

# Average policy 80%confidence and review period - 2 hour 

In [52]:
   
Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.28
nbar_s = 1.327
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

Ultab = 4 #(T+Uab)
Ultba = 5 #(T+Uba)
sigABltr = 2 #sqrt(Uab+t)
sigBAltr = 2.236 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.28
nbar_s = 1.327
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

#%%time

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if  (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = max(0,var)
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = max(0,var2)
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
MovB = []
cus_sata = []
cus_satb = []
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg80_hr2 = 0
Profit_avg80_hr2 = sum(Profit)/len(Profit)
Pmean_avg80_hr2 = 0
Pmean_avg80_hr2 = Profit[0]
cusa_avg80_hr2 = sum(cus_sata)/len(cus_sata)
cusb_avg80_hr2 = sum(cus_satb)/len(cus_satb)
sample_avg80_hr2 =  Profit[1]

16775.0
[17020, 17260, 15390, 17430]


In [53]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)
  

# Confidence interval - 90% and review period - 3 hour (S,T) policy

In [54]:
Ultab = 5 #(T+Uab)
Ultba = 6 #(T+Uba)
sigABltr = 2.236 #sqrt(Uab+t)
sigBAltr = 2.449 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.64
nbar_s = 1.661
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0


for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
              
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Sab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Sab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = var2
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Sba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Sba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = var
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                      
                                                                                                                       
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
MovB = [] 
cus_sata = []
cus_satb = []
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(sum(Profit)/len(Profit))
print(Profit)
Profit_90_hr3 = 0
Profit_90_hr3 = sum(Profit)/len(Profit)
Pmean_90_hr3 = 0
Pmean_90_hr3 = Profit[0]
cusa_90_hr3 = sum(cus_sata)/len(cus_sata)
cusb_90_hr3 = sum(cus_satb)/len(cus_satb)
sample_90_hr3  =  Profit[1]

16775.0
[17020, 17260, 15390, 17430]


In [55]:
dataframes_list

[1       Hour Station A to B Station B to A  Avail A ending  Avail A starting  \
 0          0              0              0              30                 0   
 1          0              0              0              30                 0   
 2          0              0              0              30                 0   
 3   00:00:00              5              4              25                30   
 4   01:00:00              2              3              23                25   
 5   02:00:00              4              2              19                23   
 6   03:00:00              3              1              20                23   
 7   04:00:00              1              2              22                23   
 8   05:00:00              1              5              23                24   
 9   06:00:00             17             11               7                24   
 10  07:00:00             16              9               0                 9   
 11  08:00:00             20

In [56]:
for i in range(len(dataframes_list)):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Mov a2b'] = 0
    temp_df['Mov b2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    temp_df = temp_df.rename(columns={np.nan:'Hour'})
    dataframes_list[i] = temp_df.copy(deep=True)

# Average policy 90%confidence and review period - 3 hour 

In [57]:

    
Ultab = 5 #(T+Uab)
Ultba = 6 #(T+Uba)
sigABltr = 2.236 #sqrt(Uab+t)
sigBAltr = 2.449 #sqrt(3+1)
df_mean = dataframes_list[0] 
df_mean['A2B U_DLTR'] = df_mean['Station A to B']*Ultab
df_mean['A2B Sig_DLTR'] = df_mean['Station A to B'].pow(1./2)*sigABltr 
df_mean['B2A U_DLTR'] = df_mean['Station B to A']*Ultba 
df_mean['B2A Sig_DLTR'] = df_mean['Station B to A'].pow(1./2)*sigBAltr
za = 1.64
nbar_s = 1.661
df_mean['Sab'] = df_mean['A2B U_DLTR']+(za*df_mean['A2B Sig_DLTR']) 
df_mean['Sba'] = df_mean['B2A U_DLTR']+(za*df_mean['B2A Sig_DLTR']) 
df_mean['Avg inv ab'] = df_mean['Station A to B'].div(2)+(nbar_s*df_mean['A2B Sig_DLTR'])
df_mean['Avg inv ba'] = df_mean['Station B to A'].div(2)+(nbar_s*df_mean['B2A Sig_DLTR'])
df_mean['Avg inv ab'] = df_mean['Avg inv ab'].apply(np.ceil)
df_mean['Avg inv ba'] = df_mean['Avg inv ba'].apply(np.ceil)
df_mean['Sab'] = df_mean['Sab'].apply(np.ceil)
df_mean['Sba'] = df_mean['Sba'].apply(np.ceil)
df_mean['Mov a2b'] = 0
df_mean['Mov b2a'] = 0
df_mean['Transit A'] = 0
df_mean['Transit B'] = 0

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        var=0
        var2=0
        if  (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]) and (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = max(0,var)
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = max(0,var2)
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                
        elif (temp_df['Avail A starting'].iloc[i]> df_mean['Avg inv ab'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var2 = 0
            var2 = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]-df_mean['Avg inv ab'].iloc[i]+temp_df['Mov b2a'].iloc[i-3]+temp_df['cust b'].iloc[i-3])
            temp_df['Mov a2b'].iloc[i] = max(0,var2)
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

        elif (temp_df['Avail B starting'].iloc[i]>df_mean['Avg inv ba'].iloc[i]):
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            var = 0
            var = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]-df_mean['Avg inv ba'].iloc[i]+ temp_df['Mov a2b'].iloc[i-2]+temp_df['cust a'].iloc[i-2])
            temp_df['Mov b2a'].iloc[i] = max(0,var)
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]
                                                                                                                
        else:
            temp_df['cust a'].iloc[i] = min(temp_df['Station A to B'].iloc[i],max(0,temp_df['Avail A ending'].iloc[i-1])+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3])
            temp_df['cust b'].iloc[i] = min(temp_df['Station B to A'].iloc[i],max(0,temp_df['Avail B ending'].iloc[i-1])+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2])
            temp_df['Avail A ending'].iloc[i] = max(0,temp_df['Avail A ending'].iloc[i-1]-temp_df['cust a'].iloc[i]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]-temp_df['Mov a2b'].iloc[i])  
            temp_df['Avail B ending'].iloc[i] = max(0,temp_df['Avail B ending'].iloc[i-1]-temp_df['cust b'].iloc[i]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]-temp_df['Mov b2a'].iloc[i])  
            temp_df['Avail A starting'].iloc[i] = temp_df['Avail A ending'].iloc[i-1]+temp_df['cust b'].iloc[i-3]+temp_df['Mov b2a'].iloc[i-3]
            temp_df['Avail B starting'].iloc[i] = temp_df['Avail B ending'].iloc[i-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov a2b'].iloc[i-2]
            temp_df['Transit A'].iloc[i] = temp_df['cust a'].iloc[i] + temp_df['cust a'].iloc[i-1]+temp_df['Mov a2b'].iloc[i]+temp_df['Mov a2b'].iloc[i-1]
            temp_df['Transit B'].iloc[i] = temp_df['cust b'].iloc[i] + temp_df['cust b'].iloc[i-1]+ temp_df['cust b'].iloc[i-2]+temp_df['Mov b2a'].iloc[i]+temp_df['Mov b2a'].iloc[i-1]+temp_df['Mov b2a'].iloc[i-2]

    dataframes_list[k] = temp_df.copy(deep=True)

for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)   

custA = []
custB = [] 
Profit = []
MovA = []
MovB = [] 
cus_sata = []
cus_satb = []
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov a2b'].sum()
    mb = dataframes_list[i]['Mov b2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)

print(sum(Profit)/len(Profit))
print(Profit)
Profit_avg90_hr3 = 0
Profit_avg90_hr3 = sum(Profit)/len(Profit)
Pmean_avg90_hr3 = 0
Pmean_avg90_hr3 = Profit[0]
cusa_avg90_hr3 = sum(cus_sata)/len(cus_sata)
cusb_avg90_hr3 = sum(cus_satb)/len(cus_satb)
sample_avg90_hr3  =  Profit[1]

16775.0
[17020, 17260, 15390, 17430]


In [58]:
temp_df

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,4,4,26,30,30,26,0,0,4,4,4,4,60
4,01:00:00,2,2,24,26,26,24,0,0,2,2,6,6,60
5,02:00:00,2,0,22,24,28,28,0,0,2,0,4,6,60
6,03:00:00,3,0,23,26,30,30,0,0,3,0,5,2,60
7,04:00:00,1,2,24,25,32,30,0,0,1,2,4,2,60
8,05:00:00,0,8,24,24,33,25,0,0,0,8,1,10,60
9,06:00:00,14,15,10,24,26,11,0,0,14,15,14,25,60


In [59]:
temp_df

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0
3,00:00:00,4,4,26,30,30,26,0,0,4,4,4,4,60
4,01:00:00,2,2,24,26,26,24,0,0,2,2,6,6,60
5,02:00:00,2,0,22,24,28,28,0,0,2,0,4,6,60
6,03:00:00,3,0,23,26,30,30,0,0,3,0,5,2,60
7,04:00:00,1,2,24,25,32,30,0,0,1,2,4,2,60
8,05:00:00,0,8,24,24,33,25,0,0,0,8,1,10,60
9,06:00:00,14,15,10,24,26,11,0,0,14,15,14,25,60


In [60]:
for i in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[i].copy(deep=True)
    temp_df['Avail A'] = 0
    temp_df['Avail B'] = 0
    temp_df['Avail A'][0:3] = 30
    temp_df['Avail B'][0:3] = 30
    temp_df['Mov A2b'] = 0
    temp_df['Mov B2a'] = 0
    temp_df['cust a'] = 0
    temp_df['cust b'] = 0
    temp_df['Avail A ending'] = 0
    temp_df['Avail A starting'] = 0
    temp_df['Avail B starting'] = 0
    temp_df['Avail B ending'] = 0
    temp_df['Avail A ending'][0:3] = 30
    temp_df['Avail B ending'][0:3] = 30
    temp_df['Transit A'] = 0
    temp_df['Transit B'] = 0
    temp_df['veh in sys'] = 0
    dataframes_list[i] = temp_df.copy(deep=True)
    
    
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys,Avail A,Avail B,Mov A2b,Mov B2a
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,30,30,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0,30,30,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0,30,30,0,0
3,00:00:00,6,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,01:00:00,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,02:00:00,5,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,03:00:00,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,04:00:00,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,05:00:00,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,06:00:00,19,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Future mean policy

In [61]:
mean_df = dataframes_list[0].copy(deep=True)
mean_df.loc[len(mean_df)] = 0
mean_df.loc[len(mean_df)] = 0
mean_df.loc[len(mean_df)] = 0
mean_df = mean_df.rename(columns={'Mov a2b':'Mov A2b','Mov b2a':'Mov B2a'}).copy(deep=True)

for i in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[i].copy(deep=True)
 
    for j in range(3,27,1):
        if (temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]-temp_df['cust b'].iloc[j]>mean_df['Station B to A'].iloc[j+1]+mean_df['Station B to A'].iloc[j+2]) and (temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]-temp_df['cust a'].iloc[j]<mean_df['Station A to B'].iloc[j+1]+mean_df['Station A to B'].iloc[j+2]+mean_df['Station A to B'].iloc[j+3]) and (temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]-temp_df['cust a'].iloc[j]>mean_df['Station A to B'].iloc[j+1]+mean_df['Station A to B'].iloc[j+2]+mean_df['Station A to B'].iloc[j+3]) and (temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]-temp_df['cust b'].iloc[j]<mean_df['Station B to A'].iloc[j+1]+mean_df['Station B to A'].iloc[j+2]):
            temp_df['cust a'].iloc[j] = min(max(0,temp_df['Avail A ending'].iloc[j-1])+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3],temp_df['Station A to B'].iloc[j])
            temp_df['cust b'].iloc[j] = min(max(0,temp_df['Avail B ending'].iloc[j-1])+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2],temp_df['Station B to A'].iloc[j]) 
            moveA2B = 0
            moveB2A = 0
            temp_df['Avail A starting'].iloc[j] = temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]
            temp_df['Avail B starting'].iloc[j] = temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]
            temp_df['Mov A2b'].iloc[j] = max(0,temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3] +temp_df['Mov B2a'].iloc[j-3]-temp_df['cust a'].iloc[j]-mean_df['Station A to B'].iloc[j+1] - mean_df['Station A to B'].iloc[j+2] - mean_df['Station A to B'].iloc[j+3])
            temp_df['Mov B2a'].iloc[j] = max(0,temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]-temp_df['cust b'].iloc[j]-mean_df['Station B to A'].iloc[j+1]-mean_df['Station B to A'].iloc[j+2])
            temp_df['Avail A ending'].iloc[j] = max(0,temp_df['Avail A ending'].iloc[j-1]-temp_df['cust a'].iloc[j]+temp_df['cust b'].iloc[j-3]-temp_df['Mov A2b'].iloc[j]+temp_df['Mov B2a'].iloc[j-3])
            temp_df['Avail B ending'].iloc[j] = max(0,temp_df['Avail B ending'].iloc[j-1]-temp_df['cust b'].iloc[j]+temp_df['cust a'].iloc[j-2]-temp_df['Mov B2a'].iloc[j]+temp_df['Mov A2b'].iloc[j-2])
            temp_df['Transit A'].iloc[j] = temp_df['cust a'].iloc[j] + temp_df['cust a'].iloc[j-1]+temp_df['Mov A2b'].iloc[j]+temp_df['Mov A2b'].iloc[j-1]
            temp_df['Transit B'].iloc[j] = temp_df['cust b'].iloc[j] + temp_df['cust b'].iloc[j-1]+ temp_df['cust b'].iloc[j-2]+temp_df['Mov B2a'].iloc[j]+temp_df['Mov B2a'].iloc[j-1]+temp_df['Mov B2a'].iloc[j-2]
    
        elif (temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]-temp_df['cust a'].iloc[j]>mean_df['Station A to B'].iloc[j+1]+mean_df['Station A to B'].iloc[j+2]+mean_df['Station A to B'].iloc[j+3]) and (temp_df['Avail B ending'].iloc[j-1]+temp_df['cust b'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]-temp_df['cust b'].iloc[j]<mean_df['Station B to A'].iloc[j+1]+mean_df['Station B to A'].iloc[j+2]):
            moveA2B = 0
            moveB2A = 0
            temp_df['Avail A starting'].iloc[j] = temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]
            temp_df['Avail B starting'].iloc[j] = temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[i-2]+temp_df['Mov A2b'].iloc[j-2]
            temp_df['cust a'].iloc[j] = min(max(0,temp_df['Avail A ending'].iloc[j-1])+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3],temp_df['Station A to B'].iloc[j])
            temp_df['cust b'].iloc[j] =  min(max(0,temp_df['Avail B ending'].iloc[j-1])+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2],temp_df['Station B to A'].iloc[j]) 
            temp_df['Mov A2b'].iloc[j] = max(0,temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3] +temp_df['Mov B2a'].iloc[j-3]-temp_df['cust a'].iloc[j]-mean_df['Station A to B'].iloc[j+1] - mean_df['Station A to B'].iloc[j+2] - mean_df['Station A to B'].iloc[j+3])
            temp_df['Avail A ending'].iloc[j] = max(0,temp_df['Avail A ending'].iloc[j-1]-temp_df['cust a'].iloc[j]+temp_df['cust b'].iloc[j-3]-temp_df['Mov A2b'].iloc[j]+temp_df['Mov B2a'].iloc[j-3])
            temp_df['Avail B ending'].iloc[j] = max(0,temp_df['Avail B ending'].iloc[j-1]-temp_df['cust b'].iloc[j]+temp_df['cust a'].iloc[j-2]-temp_df['Mov B2a'].iloc[j]+temp_df['Mov A2b'].iloc[j-2])
            temp_df['Transit A'].iloc[j] = temp_df['cust a'].iloc[j] + temp_df['cust a'].iloc[j-1]+temp_df['Mov A2b'].iloc[j]+temp_df['Mov A2b'].iloc[j-1]
            temp_df['Transit B'].iloc[j] = temp_df['cust b'].iloc[j] + temp_df['cust b'].iloc[j-1]+ temp_df['cust b'].iloc[j-2]+temp_df['Mov B2a'].iloc[j]+temp_df['Mov B2a'].iloc[j-1]+temp_df['Mov B2a'].iloc[j-2]
    
        elif (temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]-temp_df['cust b'].iloc[j]>mean_df['Station B to A'].iloc[j+1]+mean_df['Station B to A'].iloc[j+2]) and (temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]-temp_df['cust a'].iloc[j]<mean_df['Station A to B'].iloc[j+1]+mean_df['Station A to B'].iloc[j+2]+mean_df['Station A to B'].iloc[j+3]):
            moveA2B = 0
            moveB2A = 0
            temp_df['Avail A starting'].iloc[j] = temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]
            temp_df['Avail B starting'].iloc[j] = temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]
            temp_df['cust a'].iloc[j] = min(max(0,temp_df['Avail A ending'].iloc[j-1])+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3],temp_df['Station A to B'].iloc[j])
            temp_df['cust b'].iloc[j] =  min(max(0,temp_df['Avail B ending'].iloc[j-1])+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2],temp_df['Station B to A'].iloc[j]) 
            temp_df['Mov B2a'].iloc[j] = max(0,temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]-temp_df['cust b'].iloc[j]-mean_df['Station B to A'].iloc[j+1]-mean_df['Station B to A'].iloc[j+2])
            temp_df['Avail A ending'].iloc[j] = max(0,temp_df['Avail A ending'].iloc[j-1]-temp_df['cust a'].iloc[j]+temp_df['cust b'].iloc[j-3]-temp_df['Mov A2b'].iloc[j]+temp_df['Mov B2a'].iloc[j-3])
            temp_df['Avail B ending'].iloc[j] = max(0,temp_df['Avail B ending'].iloc[j-1]-temp_df['cust b'].iloc[j]+temp_df['cust a'].iloc[j-2]-temp_df['Mov B2a'].iloc[j]+temp_df['Mov A2b'].iloc[j-2])
            temp_df['Transit A'].iloc[j] = temp_df['cust a'].iloc[j] + temp_df['cust a'].iloc[j-1]+temp_df['Mov A2b'].iloc[j]+temp_df['Mov A2b'].iloc[j-1]
            temp_df['Transit B'].iloc[j] = temp_df['cust b'].iloc[j] + temp_df['cust b'].iloc[j-1]+ temp_df['cust b'].iloc[j-2]+temp_df['Mov B2a'].iloc[j]+temp_df['Mov B2a'].iloc[j-1]+temp_df['Mov B2a'].iloc[j-2]
      
        else:
            moveA2B = 0
            moveB2A = 0
            temp_df['Avail A starting'].iloc[j] = temp_df['Avail A ending'].iloc[j-1]+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3]
            temp_df['Avail B starting'].iloc[j] = temp_df['Avail B ending'].iloc[j-1]+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2]
            temp_df['cust a'].iloc[j] = min(max(0,temp_df['Avail A ending'].iloc[j-1])+temp_df['cust b'].iloc[j-3]+temp_df['Mov B2a'].iloc[j-3],temp_df['Station A to B'].iloc[j])
            temp_df['cust b'].iloc[j] = min(max(0,temp_df['Avail B ending'].iloc[j-1])+temp_df['cust a'].iloc[j-2]+temp_df['Mov A2b'].iloc[j-2],temp_df['Station B to A'].iloc[j]) 
            temp_df['Mov B2a'].iloc[j] = moveB2A
            temp_df['Mov A2b'].iloc[j] = moveA2B 
            temp_df['Avail A ending'].iloc[j] = max(0,temp_df['Avail A ending'].iloc[j-1]-temp_df['cust a'].iloc[j]+temp_df['cust b'].iloc[j-3]-temp_df['Mov A2b'].iloc[j]+temp_df['Mov B2a'].iloc[j-3])
            temp_df['Avail B ending'].iloc[j] = max(0,temp_df['Avail B ending'].iloc[j-1]-temp_df['cust b'].iloc[j]+temp_df['cust a'].iloc[j-2]-temp_df['Mov B2a'].iloc[j]+temp_df['Mov A2b'].iloc[j-2])
            temp_df['Transit A'].iloc[j] = temp_df['cust a'].iloc[j] + temp_df['cust a'].iloc[j-1]+temp_df['Mov A2b'].iloc[j]+temp_df['Mov A2b'].iloc[j-1]
            temp_df['Transit B'].iloc[j] = temp_df['cust b'].iloc[j] + temp_df['cust b'].iloc[j-1]+ temp_df['cust b'].iloc[j-2]+temp_df['Mov B2a'].iloc[j]+temp_df['Mov B2a'].iloc[j-1]+temp_df['Mov B2a'].iloc[j-2]
    
    dataframes_list[i] = temp_df.copy(deep=True)
    
for k in range(0,len(dataframes_list),1):
    temp_df = dataframes_list[k].copy(deep=True)
    for i in range(3,27,1):
        temp_df['veh in sys'].iloc[i] = temp_df['Avail A ending'].iloc[i]+temp_df['Transit A'].iloc[i]+temp_df['Transit B'].iloc[i]+temp_df['Avail B ending'].iloc[i]    
    
    dataframes_list[k] = temp_df.copy(deep=True)
cus_sata =[]
cus_satb =[]
Profit = []
for i in range(0,len(dataframes_list),1):
    ca = 0
    cb = 0 
    ma = 0
    mb = 0
    profit = 0
    ca = dataframes_list[i]['cust a'].sum()
    cb = dataframes_list[i]['cust b'].sum()
    da=dataframes_list[i]['Station A to B'].sum()
    db=dataframes_list[i]['Station B to A'].sum()
    csa = (ca)*100/(da)
    csb = (cb)*100/(db)
    cus_sata.append(csa)
    cus_satb.append(csb)
    ma = dataframes_list[i]['Mov A2b'].sum()
    mb = dataframes_list[i]['Mov B2a'].sum()
    pa = dataframes_list[i]['Station A to B'].sum()-dataframes_list[i]['cust a'].sum()
    pb = dataframes_list[i]['Station B to A'].sum()-dataframes_list[i]['cust b'].sum()
    profit = ca*60+cb*90-ma*20-mb*30-pa*100-pb*100
    Profit.append(profit)
    
sum(Profit)/len(Profit)
Profit_future1 = 0
Profit_future1 = sum(Profit)/len(Profit)
Pmean_future1 = 0
Pmean_future1 = Profit[0]
cusa_future1 = sum(cus_sata)/len(cus_sata)
cusb_future1 = sum(cus_satb)/len(cus_satb)
sample_future1  =  Profit[1]

In [62]:
sample_future1

18900

In [63]:
mean_df2 = mean_df.rename(columns={'Station A to B':'Mean Station A to B','Station B to A':'Mean Station B to A'}).copy(deep=True)

In [64]:
mean_df2

1,Hour,Mean Station A to B,Mean Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov A2b,Mov B2a,cust a,...,B2A U_DLTR,B2A Sig_DLTR,Sab,Sba,Avg inv ab,Avg inv ba,Avail A,Avail B,Mov A2b,Mov B2a
0,0,0,0,30,0,0,30,0,0,0,...,0,0.0,0,0,0,0,30,30,0,0
1,0,0,0,30,0,0,30,0,0,0,...,0,0.0,0,0,0,0,30,30,0,0
2,0,0,0,30,0,0,30,0,0,0,...,0,0.0,0,0,0,0,30,30,0,0
3,00:00:00,5,4,0,0,0,0,0,0,0,...,24,4.898,34,33,11,11,0,0,0,0
4,01:00:00,2,3,0,0,0,0,0,0,0,...,18,4.241792,16,25,7,9,0,0,0,0
5,02:00:00,4,2,0,0,0,0,0,0,0,...,12,3.463409,28,18,10,7,0,0,0,0
6,03:00:00,3,1,0,0,0,0,0,0,0,...,6,2.449,22,11,8,5,0,0,0,0
7,04:00:00,1,2,0,0,0,0,0,0,0,...,12,3.463409,9,18,5,7,0,0,0,0
8,05:00:00,1,5,0,0,0,0,0,0,0,...,30,5.47613,9,39,5,12,0,0,0,0
9,06:00:00,17,11,0,0,0,0,0,0,0,...,66,8.122414,101,80,24,19,0,0,0,0


In [65]:
tempref = dataframes_list[1].drop(['Mov a2b','Mov b2a'],axis=1)
tempref = tempref.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Mov A2b',
       'Mov B2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'
                                                      ])
tempref

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Mov A2b,Mov B2a,cust a,cust b,Transit A,Transit B,veh in sys
0,0,0,0,0,30,0,30,0,0,0,0,0,0,0
1,0,0,0,0,30,0,30,0,0,0,0,0,0,0
2,0,0,0,0,30,0,30,0,0,0,0,0,0,0
3,00:00:00,6,3,30,24,30,27,0,0,6,3,6,3,60
4,01:00:00,2,4,24,22,27,23,0,0,2,4,8,7,60
5,02:00:00,5,3,22,17,29,26,0,0,5,3,7,10,60
6,03:00:00,4,0,20,16,28,28,0,0,4,0,9,7,60
7,04:00:00,0,2,20,20,33,16,0,15,0,2,4,20,60
8,05:00:00,1,3,23,22,20,17,0,0,1,3,1,20,60
9,06:00:00,19,9,22,3,17,8,0,0,19,9,20,29,60


In [66]:
mergefuture = pd.DataFrame()
mergefuture = pd.merge(tempref,mean_df2[['Hour','Mean Station A to B','Mean Station B to A']])
mergefuture = mergefuture.reindex(columns=['Hour', 'Station A to B', 'Station B to A', 'Avail A starting',
       'Avail A ending', 'Avail B starting', 'Avail B ending', 'Mean Station A to B', 'Mean Station B to A', 'Mov A2b',
       'Mov B2a', 'cust a', 'cust b', 'Transit A', 'Transit B', 'veh in sys'])
mergefuture.iloc[18:,:]

1,Hour,Station A to B,Station B to A,Avail A starting,Avail A ending,Avail B starting,Avail B ending,Mean Station A to B,Mean Station B to A,Mov A2b,Mov B2a,cust a,cust b,Transit A,Transit B,veh in sys
18,00:00:00,6,3,30,24,30,27,5,4,0,0,6,3,6,3,60
19,01:00:00,2,4,24,22,27,23,2,3,0,0,2,4,8,7,60
20,02:00:00,5,3,22,17,29,26,4,2,0,0,5,3,7,10,60
21,03:00:00,4,0,20,16,28,28,3,1,0,0,4,0,9,7,60
22,04:00:00,0,2,20,20,33,16,1,2,0,15,0,2,4,20,60
23,05:00:00,1,3,23,22,20,17,1,5,0,0,1,3,1,20,60
24,06:00:00,19,9,22,3,17,8,17,11,0,0,19,9,20,29,60
25,07:00:00,14,11,20,6,9,0,16,9,0,0,14,9,33,21,60
26,08:00:00,19,6,9,0,19,13,20,7,0,0,9,6,23,24,60
27,09:00:00,21,10,9,0,27,15,18,10,0,2,9,10,18,27,60


In [67]:
Profit

[18090, 18900, 17990, 16970]

In [68]:
dataframes_list[1]

1,Hour,Station A to B,Station B to A,Avail A ending,Avail A starting,Avail B starting,Avail B ending,Mov a2b,Mov b2a,cust a,cust b,Transit A,Transit B,veh in sys,Avail A,Avail B,Mov A2b,Mov B2a
0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,30,30,0,0
1,0,0,0,30,0,0,30,0,0,0,0,0,0,0,30,30,0,0
2,0,0,0,30,0,0,30,0,0,0,0,0,0,0,30,30,0,0
3,00:00:00,6,3,24,30,30,27,0,0,6,3,6,3,60,0,0,0,0
4,01:00:00,2,4,22,24,27,23,0,0,2,4,8,7,60,0,0,0,0
5,02:00:00,5,3,17,22,29,26,0,0,5,3,7,10,60,0,0,0,0
6,03:00:00,4,0,16,20,28,28,0,0,4,0,9,7,60,0,0,0,0
7,04:00:00,0,2,20,20,33,16,0,0,0,2,4,20,60,0,0,0,15
8,05:00:00,1,3,22,23,20,17,0,0,1,3,1,20,60,0,0,0,0
9,06:00:00,19,9,3,22,17,8,0,0,19,9,20,29,60,0,0,0,0


In [69]:
#Profit results:
print("No policy :",Profit_nopol)
print("No policy mean data:",Pmean_nopol)
print("No policy sample data:",sample_nopol)
print("No policy customer satisfied at A :",cusa_nopol)
print("No policy customer satisfied at B:",cusb_nopol)
print("\n80% 1hr:",Profit_80_hr1)
print("mean data 80% 1hr:",Pmean_80_hr1)
print("80% 1hr sample data:",sample_80_hr1)
print("80% 1hr customer satisfied at A:",cusa_80_hr1)
print("80% 1hr customer satisfied at B:",cusb_80_hr1)
print("\n80% avg 1hr:",Profit_avg80_hr1)
print("mean avg data 80% 1hr:",Pmean_avg80_hr1)
print("80% avg 1hr sample data:",sample_avg80_hr1)
print("80% avg 1hr customer satisfied at A:",cusa_avg80_hr1)
print("80% avg 1hr customer satisfied at B:",cusb_avg80_hr1)
print("\n80% 2hr:",Profit_80_hr2)
print("mean data 80% 2hr:",Pmean_80_hr2)
print("80% 2hr sample data:",sample_80_hr2)
print("80% 2hr customer satisfied at A:",cusa_80_hr2)
print("80% 2hr customer satisfied at B:",cusb_80_hr2)
print("\n80% avg 2hr:",Profit_avg80_hr2)
print("mean avg data 80% 2hr:",Pmean_avg80_hr2)
print("80% avg 2hr sample data:",sample_avg80_hr2)
print("80% avg 2hr customer satisfied at A:",cusa_avg80_hr2)
print("80% avg 2hr customer satisfied at B:",cusb_avg80_hr2)
print("\n90% 1hr:",Profit_90_hr1)
print("mean data 90% 1hr:",Pmean_90_hr1)
print("90% 1hr sample data:",sample_90_hr1)
print("90% 1hr customer satisfied at A:",cusa_90_hr1)
print("90% 1hr customer satisfied at B:",cusb_90_hr1)
print("\n90% avg 1hr:",Profit_avg90_hr1)
print("mean avg data 90% 1hr:",Pmean_avg90_hr1)
print("90% avg 1hr sample data:",sample_avg90_hr1)
print("90% avg 1hr customer satisfied at A:",cusa_avg90_hr1)
print("90% avg 1hr customer satisfied at B:",cusb_avg90_hr1)
print("\n90% 2hr:",Profit_90_hr2)
print("mean data 90% 2hr:",Pmean_90_hr2)
print("90% 2hr sample data:",sample_90_hr2)
print("90% 2hr customer satisfied at A:",cusa_90_hr2)
print("90% 2hr customer satisfied at B:",cusb_90_hr2)
print("\n90% avg 2hr:",Profit_avg90_hr2)
print("mean avg data 90% 2hr:",Pmean_avg90_hr2)
print("90% avg 2hr sample data:",sample_avg90_hr2)
print("90% avg 2hr customer satisfied at A:",cusa_avg90_hr2)
print("90% avg 2hr customer satisfied at B:",cusb_avg90_hr2)
print("\n90% 3hr:",Profit_90_hr3)
print("mean data 90% 3hr:",Pmean_90_hr3)
print("90% 3hr sample data:",sample_90_hr3)
print("90% 3hr customer satisfied at A:",cusa_90_hr3)
print("90% 3hr customer satisfied at B:",cusb_90_hr3)
print("\n90% avg 3hr:",Profit_avg90_hr3)
print("mean avg data 90% 3hr:",Pmean_avg90_hr3)
print("90% avg 3hr sample data:",sample_avg90_hr3)
print("90% avg 3hr customer satisfied at A:",cusa_avg90_hr3)
print("90% avg 3hr customer satisfied at B:",cusb_avg90_hr3)
print("\n95% 1hr:",Profit_95_hr1)
print("mean data 95% 1hr:",Pmean_95_hr1)
print("95% 1hr sample data:",sample_95_hr1)
print("95% 1hr customer satisfied at A:",cusa_95_hr1)
print("95% 1hr customer satisfied at B:",cusb_95_hr1)
print("\n95% avg 1hr:",Profit_avg95_hr1)
print("mean avg data 95% 1hr:",Pmean_avg95_hr1)
print("95% avg 1hr sample data:",sample_avg95_hr1)
print("95% avg 1hr customer satisfied at A:",cusa_avg95_hr1)
print("95% avg 1hr customer satisfied at B:",cusb_avg95_hr1)
print("\n95% 2hr:",Profit_95_hr2)
print("mean data 95% 2hr:",Pmean_95_hr2)
print("95% 2hr sample data:",sample_95_hr2)
print("95% 2hr customer satisfied at A:",cusa_95_hr2)
print("95% 2hr customer satisfied at B:",cusb_95_hr2)
print("\n95% avg 2hr:",Profit_avg95_hr2)
print("mean avg data 95% 2hr:",Pmean_avg95_hr2)
print("95% avg 2hr sample data:",sample_avg95_hr2)
print("95% avg 2hr customer satisfied at A:",cusa_avg95_hr2)
print("95% avg 2hr customer satisfied at B:",cusb_avg95_hr2)
print("\nFuture1:",Profit_future1)
print("mean data future 1",Pmean_future1)
print("Future1 sample data:",sample_future1)
print("Future1 customer satisfied at A:",cusa_future1)
print("Future1 customer satisfied at B:",cusb_future1)
print("\nFuture2:",Profit_future2)
print("mean data future 2",Pmean_future2)
print("Future2 sample data:",sample_future2)
print("Future2 customer satisfied at A:",cusa_future2)
print("Future2 customer satisfied at B:",cusb_future2)


No policy : 16775.0
No policy mean data: 17020
No policy sample data: 17260
No policy customer satisfied at A : 73.34674835255267
No policy customer satisfied at B: 86.86364218448334

80% 1hr: 16775.0
mean data 80% 1hr: 17020
80% 1hr sample data: 17260
80% 1hr customer satisfied at A: 73.34674835255267
80% 1hr customer satisfied at B: 86.86364218448334

80% avg 1hr: 16775.0
mean avg data 80% 1hr: 17020
80% avg 1hr sample data: 17260
80% avg 1hr customer satisfied at A: 73.34674835255267
80% avg 1hr customer satisfied at B: 86.86364218448334

80% 2hr: 16775.0
mean data 80% 2hr: 17020
80% 2hr sample data: 17260
80% 2hr customer satisfied at A: 73.34674835255267
80% 2hr customer satisfied at B: 86.86364218448334

80% avg 2hr: 16775.0
mean avg data 80% 2hr: 17020
80% avg 2hr sample data: 17260
80% avg 2hr customer satisfied at A: 73.34674835255267
80% avg 2hr customer satisfied at B: 86.86364218448334

90% 1hr: 16775.0
mean data 90% 1hr: 17020
90% 1hr sample data: 17260
90% 1hr customer sa

NameError: name 'Profit_future2' is not defined